<a href="https://colab.research.google.com/github/maing-rv/python_cw/blob/main/python_cw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collab code environment:
Mai was here.

Problem statement and hypothesis: 


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline